In [30]:
# GPU 지원하는지 확인
import tensorflow as tf
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

# 심층 신경망

In [31]:
# 입력층, 은닉층, 출력층 세가지가 있지만 입력층은 생각하지 않는다
# 은닉층의 개수는 딱히 정해진 것은 없다
# 출력층 10개의 확률을 계산해야하기 때문에 10개보다는 많아야 할 것
# 활성화 함수를 거쳐서 비선형 함수로 데이터를 변형해서 단순히 선형식이 합쳐지지 않도록 해야한다.
# 활성화함수... 다시 찾아봐야겠습니다.

from tensorflow import keras

# 데이터 삽입
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28*28)

from sklearn.model_selection import train_test_split
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)

In [32]:
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,))
dense2 = keras.layers.Dense(10, activation='softmax')
model = keras.Sequential([dense1, dense2])

In [33]:
model.summary()
# 케라스 fit 에서 batch size를 정할 수 있는데, 기본값 32
# 배치... 어쩌구 하는거랑 활성화함수.. 다시 봐야겠다.

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_38 (Dense)            (None, 100)               78500     
                                                                 
 dense_39 (Dense)            (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [34]:
# 층을 추가하는 다른 방법
model = keras.Sequential([
    keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)),
    keras.layers.Dense(10, activation='softmax')
], name='패션 MNIST 모델')

# 세번째 방법
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(10, activation='softmax'))
model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_42 (Dense)            (None, 100)               78500     
                                                                 
 dense_43 (Dense)            (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [35]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 1s 705us/step - loss: 0.5662 - accuracy: 0.8060
Epoch 2/5
1500/1500 [==============================] - 1s 639us/step - loss: 0.4099 - accuracy: 0.8534
Epoch 3/5
1500/1500 [==============================] - 1s 650us/step - loss: 0.3752 - accuracy: 0.8663
Epoch 4/5
1500/1500 [==============================] - 1s 758us/step - loss: 0.3530 - accuracy: 0.8722
Epoch 5/5
1500/1500 [==============================] - 1s 713us/step - loss: 0.3358 - accuracy: 0.8781


## 몇개 더 해볼까?

In [36]:
model = keras.Sequential()
model.add(keras.layers.Dense(500, activation='sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(200, activation='sigmoid'))
model.add(keras.layers.Dense(100, activation='sigmoid'))
model.add(keras.layers.Dense(10, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.6036 - accuracy: 0.7818
Epoch 2/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.4115 - accuracy: 0.8509
Epoch 3/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3699 - accuracy: 0.8658
Epoch 4/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3426 - accuracy: 0.8766
Epoch 5/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3231 - accuracy: 0.8826


## 렐루 함수와 Flatten 층

In [37]:
model = keras.Sequential()
# 28x28을 1차원으로 풀어서 제공해줬었는데, 그것을 Flatten 클래스가 해준다.
model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_6 (Flatten)         (None, 784)               0         
                                                                 
 dense_48 (Dense)            (None, 100)               78500     
                                                                 
 dense_49 (Dense)            (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


## 옵티마이저
compile() 메서드의 기본 경사 하강법 알고리즘은 RMSprop  
다른 경사 하강법 알고리즘을 적용해보자.

In [38]:
# 데이터 삽입
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

train_scaled = train_input / 255.0
# flatten 층 덕분에 reshape를 하지 않아도 된다.
# train_scaled = train_scaled.reshape(-1, 28*28)

from sklearn.model_selection import train_test_split
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)

In [39]:
# sgd 알고리즘을 적용하는 방법
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics='accuracy')

# 객체를 만드는 다른 방법
sgd = keras.optimizers.SGD()
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics='accuracy')

# 위의 방법대로 하면 옵션을 넣어줄 수 있다.
sgd = keras.optimizers.SGD(learning_rate=0.1)
sgd = keras.optimizers.SGD(momentum=0.9, nesterov=True)

In [40]:
# Adam 옵티마이저를 사용해보자
model = keras.Sequential()
# 28x28을 1차원으로 풀어서 제공해줬었는데, 그것을 Flatten 클래스가 해준다.
model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

model.summary()

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_7 (Flatten)         (None, 784)               0         
                                                                 
 dense_50 (Dense)            (None, 100)               78500     
                                                                 
 dense_51 (Dense)            (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
1500/1500 [==============================] - 1s 698us/step - loss: 0.5253 - accuracy: 0.8187
Epoch 2/5
1500/1500 [==============================] - 1s 573us/step - loss: 0.3970 - accuracy: 0.8601
Epoch 3/5
1500/1500 [==============================] - 1s 541us/step - loss: 0.3570 - accuracy: 0.8705
Epoc

In [42]:
# 검증
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 0s 385us/step - loss: 0.3527 - accuracy: 0.8719


[0.35271358489990234, 0.871916651725769]